# Project test

In this assignment, we are going to improve our codes in PattRecClasses and implement forward algorithm inside MarkovChain code as well as functions such as logprob and prob in Guassian in order to generate proper input values for forward algorithm.

In [1]:
from PattRecClasses import HMM_TA 
import scipy.stats
from matplotlib import pyplot as plt
# For the code to work you might have to pip install scipy

from CharacterFeatureExtractor import featureExtractor
from DrawCharacter import DrawCharacter

import numpy as np

import pandas as pd


In [2]:
'''
Multivariate Gaussian Distribution Class
'''
class multigaussD:
    mean = np.array([0])
    cov = np.array([[0]])
    def __init__(self, mu, C):
        if C.shape[0] is not C.shape[1]:
            print("error, non-square covariance matrix supplied")
            return
        if mu.shape[0] is not C.shape[0]:
            print("error, mismatched mean vector and covariance matrix dimensions")
            return
        self.mean = mu
        if np.where(np.diag(C)==0)[0].shape[0] != 0:
            C += np.diagflat(np.ones(C.shape[0])/10000)
        C[np.isnan(C)]=1
        self.cov = C
        return
    def random(self, num):
        return np.random.multivariate_normal(self.mean, self.cov, num)
    def rand(self):
        return np.random.multivariate_normal(self.mean, self.cov, 1)[0]
    def likelihood(self, X):
        p = scipy.stats.multivariate_normal(self.mean, self.cov, 1)
        pd = p.pdf(X)
        return pd
    def loghood(self, X):
        return np.log(self.likelihood(X))
    def getmean(self):
        return self.mean
    def getcov(self):
        return self.cov
    
def prob(x, B):
    T = x.shape[0]
    N = B.shape[0]
    res = np.zeros((T, N))
    for i in range(T):
        for j in range(N):
            res[i,j] = B[j].likelihood(x[i])
    scaled = np.zeros(res.shape)
    for i in range(scaled.shape[0]):
        for j in range(scaled.shape[1]):
            scaled[i, j] = res[i,j]/np.amax(res[i])
    return res, scaled


def logprob(x, B):
    res, scaled = prob(x,B)
    return np.log(res), np.log(scaled)

# Code skeleton

In [3]:
### data prep
db_name = "database_test"
data_features = pd.read_pickle(r'data/'+db_name+'_features.cdb')
data_labels = pd.read_pickle(r'data/'+db_name+'_labels.cdb')

# data_features[k][r] == np.array (ndim, t); K (number of letters) of R samples with Tr individual lengths
print((data_features[1][1].shape))
print(data_labels)

# Works! But test on easier features

### train for one character (seen from labels)


##### TESTING FOR X, char = 4

char = 4
obs = data_features[char]

# obsTA = np.array([ hm_learn.rand(100)[0] for _ in range(10) ])
# print(type(obsTA))
# print(obsTA[1].shape) == (100,2)
# Our data has format (2,15) ! Transpose all datapoints
for i in range(len(obs)):
    obs[i] = np.transpose(obs[i])
    
print(len(obs))
print(obs[len(obs)-1].shape)
print(type(obs))
print(obs[1])


### training

# Estimate the HMM parameters from the obseved samples
# Start by. assigning initial HMM parameter values,
# then refine these iteratively

### These values are generally true for most of our two-stroke characters
qstar = np.array([1, 0])
Astar = np.array([[0.9, 0.1], [0, 1]])

### Values chosen for X from FeatureVisualization
f1s1mean=10
f1s2mean=45
f2s1mean=-60
f2s2mean=70

meansstar = np.array( [[f1s1mean, f2s1mean], [f1s2mean, f2s2mean]] )

covsstar  = np.array( [[[1, 1],[1, 1]], 
                       [[1, 1],[1,1]]] )

Bstar = np.array([multigaussD(meansstar[0], covsstar[0]),
                  multigaussD(meansstar[1], covsstar[1])])


hm_learn = HMM_TA.HMM(qstar, Astar, Bstar)

print("Running the Baum Welch Algorithm...")
hm_learn.baum_welch(obs, 20, prin=1, uselog=False)

(2, 15)
['A', 'C', 'K', 'P', 'X']
5
(23, 2)
<class 'list'>
[[ 12.370322   -75.96076991]
 [ 12.08405109 -65.5530586 ]
 [ 12.20756073 -55.00499318]
 [  9.22054958 -49.39571873]
 [ 12.0425997  -48.36347404]
 [ 10.81765895 -56.30694585]
 [ 10.00100512 -53.12711573]
 [ 10.81765895 -56.30694585]
 [ 10.00100512 -53.12711573]
 [ 12.20756073 -55.00499318]
 [  8.94527703 -63.4319622 ]
 [  9.22054958 -49.39571873]
 [ 10.81765895 -56.30694585]
 [  9.43498625 -57.99163017]
 [  7.07207293  81.87288427]
 [ 35.86632721  83.71609326]
 [ 39.61191323  56.31291909]
 [ 38.22924053  57.99760341]
 [ 38.01480386  49.40169198]
 [ 39.42540521  48.81706145]
 [ 43.2174645   56.31291909]
 [ 40.83685398  48.36944728]
 [ 40.83685398  48.36944728]
 [ 38.01480386  40.60428127]
 [ 38.01480386  49.40169198]
 [ 37.39758467  54.46530883]
 [ 37.28054077  45.00298662]
 [ 38.01480386  40.60428127]
 [ 38.01480386  49.40169198]
 [ 38.7952594   53.13308897]
 [ 36.60550908  50.19741553]
 [ 36.60550908  39.80855771]]
Running the 

/home/jonas/Local/Pattern-Recognition-Project/PattRecClasses/HMM_TA.py:141: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  gammas = np.array(gammas)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# TA test

In [ ]:
# Define a HMM
q = np.array([0.8, 0.2])
A = np.array([[0.95, 0.05],
            [0.30, 0.70]])

means = np.array( [[0, 0], [2, 2]] )
covs  = np.array( [[[1, 2],[2, 4]], 
                [[1, 0],[0, 3]]] )

B = np.array([multigaussD(means[0], covs[0]),
            multigaussD(means[1], covs[1])])


hm  = HMM_TA.HMM(q, A, B)


# Test that baum_welch works with inputs on the same format as us
obs = []  
for i in range(10): 
    obs += [hm.rand(100)[0]]

print(obs[1].shape)
print(obs[1])
print(type(obs))

print('True HMM parameters:')
print('q:')
print(q)
print('A:')
print(A)
print('B: means, covariances')
print(means)
print(covs)

In [ ]:
# Estimate the HMM parameters from the obseved samples
# Start by. assigning initial HMM parameter values,
# then refine these iteratively
qstar = np.array([0.8, 0.2])
Astar = np.array([[0.5, 0.5], [0.5, 0.5]])

meansstar = np.array( [[0, 0], [0, 0]] )

covsstar  = np.array( [[[1, 0],[0, 1]], 
                       [[1, 0],[0,1]]] )

Bstar = np.array([multigaussD(meansstar[0], covsstar[0]),
                  multigaussD(meansstar[1], covsstar[1])])


hm_learn = HMM_TA.HMM(qstar, Astar, Bstar)

print("Running the Baum Welch Algorithm...")
hm_learn.baum_welch(obs, 20, prin=1, uselog=False)

# obs is in the format:
# obsTA = np.array([ hm.rand(100)[0] for _ in range(10) ])
# ; shape (10 2 100)

# test
# Test the Viterbi algorithm
print("Running the Viterbi Algorithm...")
obs, true_states = hm.rand(100)

print("True States:\n",true_states)
print("Predicted States:\n", hm_learn.viterbi(obs))

####### SOMETIMES ALL RESULTS ARE TRANSPOSED/INVERTED MATRIXES!!!
####### and sometimes it just works. Great stuff

# descision

# P test

In [ ]:
#Get symbol-1 (small p on top left quadrant)
dc1 = np.load("data/P_top_left.npy")

#Get symbol-2 (small p on bottom right quadrant)
dc2 = np.load("data/P_bottom_right.npy")

#Get symbol-3(twice as big p filling entire window)
dc3 = np.load("data/P_big.npy")

thr = 8 # threshold for sampling and distance normalization

# #Feature vectors are returned
feature_symbol1, sampled_symbol1 = featureExtractor(dc1,thr,False)
feature_symbol2, sampled_symbol2 = featureExtractor(dc2,thr,False)
feature_symbol3, sampled_symbol3 = featureExtractor(dc3,thr,False)


# normalized distance ,slope, and t for symbol-1
f1_symbol1 = feature_symbol1[0]
f2_symbol1 = feature_symbol1[1]
t1 = np.array(range(0,feature_symbol1.shape[1]))


# normalized distance ,slope, and t for symbol-2
f1_symbol2 = feature_symbol2[0]
f2_symbol2 = feature_symbol2[1]
t2 = np.array(range(0,feature_symbol2.shape[1]))

# normalized distance ,slope, and t for symbol-2
f1_symbol3 = feature_symbol3[0]
f2_symbol3 = feature_symbol3[1]
t3 = np.array(range(0,feature_symbol3.shape[1]))

f, axarr = plt.subplots(3, 3)
f.suptitle('Scale & Position Effect', fontsize=20)


#------------- SYMBOL DRAWINGS
#Drawing of sampled symbol-1
axarr[0, 0].scatter(sampled_symbol1[0], sampled_symbol1[1])
axarr[0, 0].set(xlabel = "X-Coordinate", ylabel = "Y-Coordinate")
axarr[0, 0].set_title('Symbol-1')
axarr[0, 0].set_xlim([0,210])
axarr[0, 0].set_ylim([0,210])

#Drawing of sampled symbol-2
axarr[0, 1].scatter(sampled_symbol2[0], sampled_symbol2[1])
axarr[0, 1].set(xlabel = "X-Coordinate", ylabel = "Y-Coordinate")
axarr[0, 1].set_title('Symbol-2')
axarr[0, 1].set_xlim([0,210])
axarr[0, 1].set_ylim([0,210])

#Drawing of sampled symbol-3
axarr[0, 2].scatter(sampled_symbol3[0], sampled_symbol3[1])
axarr[0, 2].set(xlabel = "X-Coordinate", ylabel = "Y-Coordinate")
axarr[0, 2].set_title('Symbol-3')
axarr[0, 2].set_xlim([0,210])
axarr[0, 2].set_ylim([0,210])

#------------- ABSOLUTE DISTANCE FEATURE

#Absolute distance plot of symbol-1
axarr[1, 0].plot(t1, f1_symbol1)
axarr[1, 0].set(xlabel = "Time", ylabel = "Normalized Distance")
axarr[1, 0].set_ylim([0,np.max(f1_symbol1)])

#Absolute distance plot of symbol-2
axarr[1, 1].plot(t2, f1_symbol2)
axarr[1, 1].set(xlabel = "Time", ylabel = "Normalized Distance")
axarr[1, 1].set_ylim([0,np.max(f1_symbol2)])

#Absolute distance plot of symbol-3
axarr[1, 2].plot(t3, f1_symbol3)
axarr[1, 2].set(xlabel = "Time", ylabel = "Normalized Distance")
axarr[1, 2].set_ylim([0,np.max(f1_symbol3)])

#------------- SLOPE FEATURE

#Y-wise distance plot of symbol-1
axarr[2, 0].plot(t1, f2_symbol1)
axarr[2, 0].set(xlabel = "Time", ylabel = "Slope(Degrees)")
axarr[2, 0].set_ylim([-120,120])

#Y-wise distance plot of symbol-2
axarr[2, 1].plot(t2, f2_symbol2)
axarr[2, 1].set(xlabel = "Time", ylabel = "Slope(Degrees)")
axarr[2, 1].set_ylim([-120,120])

#Y-wise distance plot of symbol-3
axarr[2, 2].plot(t3, f2_symbol3)
axarr[2, 2].set(xlabel = "Time", ylabel = "Slope(Degrees)")
axarr[2, 2].set_ylim([-120,120])

plt.savefig('fig/P_test.png', bbox_inches='tight', dpi = 300)
